In [2]:
%matplotlib inline

import librosa
from scipy.fftpack import dct, idct

1. [Spectrograms in Python](https://timsainb.github.io/spectrograms-mfccs-and-inversion-in-python.html)

2. [Audio Tools in Python](https://gist.github.com/kastnerkyle/179d6e9a88202ab0a2fe)